<a href="https://colab.research.google.com/github/stas420/machine_learning_ev_ice/blob/main/src/EV_to_ICE_ML.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Setup and dependencies
Run the below cell, which shall clone the repo contents and prepare the machine for work. This may take a few minutes.

***Important:*** This is the first cell you shall run in this project, otherwise it won't be working.

After 'pip' is done, it may ask you to restart this session - please do accordingly.

In [1]:
# remove possible leftovers and clear-clone the whole repo
!rm -rf ./machine_learning_ev_ice
!git clone https://github.com/stas420/machine_learning_ev_ice.git

# a git pull pro forma
%cd ./machine_learning_ev_ice
!git pull

# go to our main directory
%cd ./src

# check (and download, if needed) dependencies for the project
%pip install -r requirements.txt

Cloning into 'machine_learning_ev_ice'...
remote: Enumerating objects: 38, done.
remote: Counting objects: 100% (38/38), done.
remote: Compressing objects: 100% (28/28), done.
remote: Total 38 (delta 12), reused 29 (delta 7), pack-reused 0 (from 0)
Receiving objects: 100% (38/38), 9.14 KiB | 9.14 MiB/s, done.
Resolving deltas: 100% (12/12), done.
/content/machine_learning_ev_ice
Already up to date.
/content/machine_learning_ev_ice/src


# test run to check labelImg

In [2]:
# Launch labelImg
!labelImg

qt.qpa.plugin: Could not load the Qt platform plugin "xcb" in "" even though it was found.
This application failed to start because no Qt platform plugin could be initialized. Reinstalling the application may fix this problem.

Available platform plugins are: eglfs, linuxfb, minimal, minimalegl, offscreen, vnc, wayland-egl, wayland, wayland-xcomposite-egl, wayland-xcomposite-glx, webgl, xcb.



# test run to check torch

In [3]:
import torch

rand = torch.rand(5, 3)
print(rand)

tensor([[0.9320, 0.5988, 0.1856],
        [0.5191, 0.3469, 0.9685],
        [0.1125, 0.9076, 0.0556],
        [0.0212, 0.8824, 0.7880],
        [0.8299, 0.5203, 0.5885]])


Dżem dobry
:)